We would like to give awards to the best wineries. Come up with 3 relevant ones. Which wineries should we choose and why? Be creative ;)

In [4]:
# Import required libraries
import sqlite3
  
# Connect to SQLite database
conn = sqlite3.connect(r'../data/vivino.db')

# Create a cursor object
cur = conn.cursor()

In [126]:
cur.execute("""
            SELECT *
            FROM wineries 
            WHERE wineries.id = 1574
            ;
            """)
cur.fetchall()

[]

In [128]:

cur.execute("""
            SELECT wines.name, wines.winery_id, wineries.id, wineries.name
            FROM wines
                join wineries on wines.winery_id = wineries.id
            LIMIT 15;
            """)
cur.fetchall()

[('The Armagh Shiraz', 1651, 1651, "Tenuta Tignanello 'Solaia'"),
 ('Colheita Porto', 1652, 1652, 'Tignanello'),
 ('Lion Tamer Napa Valley Red Blend', 1471, 1471, 'Siepi'),
 ('Raut Lagrein', 75712, 75712, 'Corte di Cama Sforzato di Valtellina')]

In [77]:

cur.execute("""
            SELECT  w.name, count(v.id)
            FROM wines w
            JOIN vintages v ON v.wine_id = w.id
            GROUP BY w.name
            LIMIT 10;
            """)
cur.fetchall()


[("'Giulio Ferrari' Riserva del Fondatore", 10),
 ("'Giulio Ferrari' Riserva del Fondatore Rosé", 1),
 ('110 e Lode Merlot', 1),
 ('123 Brunello di Montalcino Riserva', 1),
 ('1694 Classification Red', 1),
 ('25 anni Montefalco Sagrantino', 1),
 ('30 Year Old Tawny Port', 1),
 ('40 Anos Tawny Port', 1),
 ('40 Years Old Port', 1),
 ('40 Years Old Tawny Port', 1)]

In [4]:
cur.execute("""
            SELECT tl.country_code, tl.name,vtr.rank 
            FROM toplists tl
            JOIN vintage_toplists_rankings vtr ON tl.id = vtr.top_list_id
            WHERE tl.name LIKE '%2020%' and tl.name LIKE '%Red%'
            LIMIT 15;
            """)
cur.fetchall()

[('global', "Vivino's 2020 Wine Style Awards: Tuscan Red", 1),
 ('global', "Vivino's 2020 Wine Style Awards: Tuscan Red", 2),
 ('global',
  "Vivino's 2020 Wine Style Awards: Southern Rhône Châteauneuf-du-Pape Red",
  1),
 ('global', "Vivino's 2020 Wine Style Awards: Northern Italy Red", 3),
 ('global',
  "Vivino's 2020 Wine Style Awards: Spanish Ribera Del Duero Red",
  3),
 ('global', "Vivino's 2020 Wine Style Awards: Tuscan Red", 10),
 ('global', "Vivino's 2020 Wine Style Awards: Tuscan Red", 5),
 ('global', "Vivino's 2020 Wine Style Awards: Tuscan Red", 8),
 ('global', "Vivino's 2020 Wine Style Awards: Languedoc-Roussillon Red", 1),
 ('global', "Vivino's 2020 Wine Style Awards: Southern Rhône Red", 1),
 ('global', "Vivino's 2020 Wine Style Awards: Australian Rhône Blend Red", 1)]

In [ ]:
cur.execute("""
            SELECT  DISTINCT country_code
            FROM toplists
            LIMIT 15;
            """)
cur.fetchall()

We have detected that a big cluster of customers like a specific combination of tastes. We have identified a few primary keywords that match this. We would like you to find all the wines that have those keywords. To ensure the accuracy of our selection, ensure that more than 10 users confirmed those keywords. Also, identify the group_name related to those keywords.
- coffee
- toast
- green apple
- cream
- citrus

In [60]:
cur.execute("""
            SELECT w.id, w.name, k.name, kw.group_name, sum(kw.count) 
            FROM wines w
            JOIN keywords_wine kw ON kw.wine_id = w.id
            JOIN keywords k ON k.id = kw.keyword_id
            WHERE k.name in ('coffee','toast','green apple','cream','citrus')
                and kw.count > 10
                and kw.keyword_type = 'primary'
            GROUP BY w.id, w.name, k.name, kw.group_name
            ;
            """)

import csv

result = cur.fetchall()

c = csv.writer(open("../data/temp.csv","w"))
c.writerows(result)




In [77]:
import pandas as pd

cur.execute("""
            SELECT w.name, k.name, kw.group_name, sum(kw.count) 
            FROM wines w
            JOIN keywords_wine kw ON kw.wine_id = w.id
            JOIN keywords k ON k.id = kw.keyword_id
            WHERE w.id in (
                SELECT
                w.id
                FROM wines w
                JOIN keywords_wine kw ON kw.wine_id = w.id
                JOIN keywords k ON k.id = kw.keyword_id
                WHERE k.name in ('coffee','toast','green apple','cream','citrus')
                    and kw.count > 10
                    and kw.keyword_type = 'primary'
                GROUP BY w.id
                HAVING MAX(CASE WHEN k.name = 'coffee' THEN kw.group_name ELSE 0 END) <> 0 AND 
                    MAX(CASE WHEN k.name = 'toast' THEN kw.group_name ELSE 0 END) <> 0 AND 
                    MAX(CASE WHEN k.name = 'green apple' THEN kw.group_name ELSE 0 END) <> 0 AND 
                    MAX(CASE WHEN k.name = 'cream' THEN kw.group_name ELSE 0 END) <> 0 AND 
                    MAX(CASE WHEN k.name = 'citrus' THEN kw.group_name ELSE 0 END) <> 0
            ) and k.name in ('coffee','toast','green apple','cream','citrus')
                    and kw.count > 10
                    and kw.keyword_type = 'primary'
            GROUP BY w.name, k.name, kw.group_name
            ;
            """)

# Put it all to a data frame
sql_data = pd.DataFrame(cur.fetchall(), columns=['name_wine', 'name_keywords', 'group_name', 'count'])

display(sql_data)


,name_wine,name_keywords,group_name,count
0,Belle Epoque Brut Champagne,citrus,citrus_fruit,423
1,Belle Epoque Brut Champagne,coffee,oak,14
2,Belle Epoque Brut Champagne,cream,microbio,207
3,Belle Epoque Brut Champagne,green apple,tree_fruit,224
4,Belle Epoque Brut Champagne,toast,non_oak,239
...,...,...,...,...
80,Vintage,citrus,citrus_fruit,406
81,Vintage,coffee,oak,56
82,Vintage,cream,microbio,240
83,Vintage,green apple,tree_fruit,96


In [74]:
import pandas as pd

cur.execute("""
            SELECT
            w.id,
            w.name,
            MAX(CASE WHEN k.name = 'coffee' THEN kw.group_name ELSE 0 END) AS 'coffee',
            MAX(CASE WHEN k.name = 'toast' THEN kw.group_name ELSE 0 END) AS 'toast',
            MAX(CASE WHEN k.name = 'green apple' THEN kw.group_name ELSE 0 END) AS 'green_apple',
            MAX(CASE WHEN k.name = 'cream' THEN kw.group_name ELSE 0 END) AS 'cream',
            MAX(CASE WHEN k.name = 'citrus' THEN kw.group_name ELSE 0 END) AS 'citrus',
            sum(kw.count) AS countUsers
            FROM wines w
            JOIN keywords_wine kw ON kw.wine_id = w.id
            JOIN keywords k ON k.id = kw.keyword_id
            WHERE k.name in ('coffee','toast','green apple','cream','citrus')
                and kw.count > 10
                and kw.keyword_type = 'primary'
            GROUP BY w.id, w.name
            HAVING coffee <> 0 AND toast <> 0 AND green_apple <> 0 AND cream <> 0 AND citrus <> 0
            ;
            """)

# Put it all to a data frame
sql_data = pd.DataFrame(cur.fetchall(), columns=['id', 'name_wine', 'group_name_coffee', 'group_name_toast', 'group_name_green_apple', 'group_name_cream', 'group_name_citrus', 'count'])

display(sql_data)

,id,name_wine,group_name_coffee,group_name_toast,group_name_green_apple,group_name_cream,group_name_citrus,count
0,18931,La Grande Année Brut Champagne,oak,non_oak,tree_fruit,microbio,citrus_fruit,1611
1,74304,Cristal Brut Champagne (Millésimé),oak,non_oak,tree_fruit,microbio,citrus_fruit,2238
2,79162,Belle Epoque Brut Champagne,oak,non_oak,tree_fruit,microbio,citrus_fruit,1107
3,79631,Vintage,oak,non_oak,tree_fruit,microbio,citrus_fruit,1154
4,79836,La Grande Dame Brut Champagne,oak,non_oak,tree_fruit,microbio,citrus_fruit,875
5,86684,Brut Champagne,oak,non_oak,tree_fruit,microbio,citrus_fruit,7363
6,1105696,Trebbiano d'Abruzzo,oak,non_oak,tree_fruit,microbio,citrus_fruit,128
7,1127349,Le Mesnil Blanc de Blancs (Cuvée S) Brut Champ...,oak,non_oak,tree_fruit,microbio,citrus_fruit,546
8,1136950,Comtes de Champagne Blanc de Blancs,oak,non_oak,tree_fruit,microbio,citrus_fruit,2184
9,1153863,Sauternes,oak,non_oak,tree_fruit,microbio,citrus_fruit,504


We would like to select wines that are easy to find all over the world. Find the top 3 most common grape all over the world and for each grape, give us the the 5 best rated wines.

In [36]:
import pandas as pd

cur.execute("""
            SELECT g.id, g.name, count(mg.country_code) AS country, sum(mg.wines_count) AS winescount
            FROM grapes g
            JOIN most_used_grapes_per_country mg ON mg.grape_id = g.id
            GROUP BY g.id, g.name
            ORDER BY country desc
            LIMIT 3
            ;
            """)

# Put it all to a data frame
sql_data = pd.DataFrame(cur.fetchall(), columns=['id', 'name', 'country', 'wines_count'])

display(sql_data)

,id,name,country,wines_count
0,2,Cabernet Sauvignon,12,9621012
1,10,Merlot,11,6233909
2,5,Chardonnay,6,3625248


In [47]:
import pandas as pd

cur.execute("""
            SELECT w.name, w.ratings_average, w.ratings_count, g.name
            FROM wines w
            JOIN regions r ON r.id = w.region_id
            JOIN countries c ON c.code = r.country_code
            JOIN most_used_grapes_per_country mg ON mg.country_code = c.code
            JOIN grapes g ON mg.grape_id = g.id
            WHERE mg.grape_id = 2 and w.ratings_count > 2000
            ORDER BY w.ratings_average DESC, w.ratings_count DESC
            LIMIT 5
            ;
            """)

# Put it all to a data frame
sql_data = pd.DataFrame(cur.fetchall(), columns=['wine', 'ratings_average', 'ratings_count', 'grape'])


cur.execute("""
            SELECT w.name, w.ratings_average, w.ratings_count, g.name
            FROM wines w
            JOIN regions r ON r.id = w.region_id
            JOIN countries c ON c.code = r.country_code
            JOIN most_used_grapes_per_country mg ON mg.country_code = c.code
            JOIN grapes g ON mg.grape_id = g.id
            WHERE mg.grape_id = 5 and w.ratings_count > 2000
            ORDER BY w.ratings_average DESC, w.ratings_count DESC
            LIMIT 5
            ;
            """)
sql_data2 = pd.DataFrame(cur.fetchall(), columns=['wine', 'ratings_average', 'ratings_count', 'grape'])

cur.execute("""
            SELECT w.name, w.ratings_average, w.ratings_count, g.name
            FROM wines w
            JOIN regions r ON r.id = w.region_id
            JOIN countries c ON c.code = r.country_code
            JOIN most_used_grapes_per_country mg ON mg.country_code = c.code
            JOIN grapes g ON mg.grape_id = g.id
            WHERE mg.grape_id = 10 and w.ratings_count > 2000
            ORDER BY w.ratings_average DESC, w.ratings_count DESC
            LIMIT 5
            ;
            """)
sql_data3 = pd.DataFrame(cur.fetchall(), columns=['wine', 'ratings_average', 'ratings_count', 'grape'])
     
sql_dataTotal = pd.concat([sql_data,sql_data2, sql_data3])


display(sql_dataTotal)

,wine,ratings_average,ratings_count,grape
0,Cabernet Sauvignon,4.8,2941,Cabernet Sauvignon
1,Unico,4.7,45140,Cabernet Sauvignon
2,Special Selection Cabernet Sauvignon,4.7,41236,Cabernet Sauvignon
3,Grange,4.7,24356,Cabernet Sauvignon
4,Unico Reserva Especial Edición,4.7,13025,Cabernet Sauvignon
0,Cabernet Sauvignon,4.8,2941,Chardonnay
1,Sauternes,4.7,44126,Chardonnay
2,Special Selection Cabernet Sauvignon,4.7,41236,Chardonnay
3,Pomerol,4.7,32157,Chardonnay
4,Grange,4.7,24356,Chardonnay


In [49]:
import pandas as pd

cur.execute("""
            SELECT w.name, w.ratings_average, w.ratings_count, g.name
            FROM wines w
            JOIN regions r ON r.id = w.region_id
            JOIN countries c ON c.code = r.country_code
            JOIN most_used_grapes_per_country mg ON mg.country_code = c.code
            JOIN grapes g ON mg.grape_id = g.id
            WHERE mg.grape_id = 2 and w.ratings_count > 500
            ORDER BY w.ratings_average DESC, w.ratings_count DESC
            LIMIT 5
            ;
            """)

# Put it all to a data frame
sql_data = pd.DataFrame(cur.fetchall(), columns=['wine', 'ratings_average', 'ratings_count', 'grape'])


cur.execute("""
            SELECT w.name, w.ratings_average, w.ratings_count, g.name
            FROM wines w
            JOIN regions r ON r.id = w.region_id
            JOIN countries c ON c.code = r.country_code
            JOIN most_used_grapes_per_country mg ON mg.country_code = c.code
            JOIN grapes g ON mg.grape_id = g.id
            WHERE mg.grape_id = 5 and w.ratings_count > 500
            ORDER BY w.ratings_average DESC, w.ratings_count DESC
            LIMIT 5
            ;
            """)
sql_data2 = pd.DataFrame(cur.fetchall(), columns=['wine', 'ratings_average', 'ratings_count', 'grape'])

cur.execute("""
            SELECT w.name, w.ratings_average, w.ratings_count, g.name
            FROM wines w
            JOIN regions r ON r.id = w.region_id
            JOIN countries c ON c.code = r.country_code
            JOIN most_used_grapes_per_country mg ON mg.country_code = c.code
            JOIN grapes g ON mg.grape_id = g.id
            WHERE mg.grape_id = 10 and w.ratings_count > 500
            ORDER BY w.ratings_average DESC, w.ratings_count DESC
            LIMIT 5
            ;
            """)
sql_data3 = pd.DataFrame(cur.fetchall(), columns=['wine', 'ratings_average', 'ratings_count', 'grape'])
     
sql_dataTotal = pd.concat([sql_data,sql_data2, sql_data3])


display(sql_dataTotal)

,wine,ratings_average,ratings_count,grape
0,Cabernet Sauvignon,4.8,2941,Cabernet Sauvignon
1,Unico,4.7,45140,Cabernet Sauvignon
2,Special Selection Cabernet Sauvignon,4.7,41236,Cabernet Sauvignon
3,Grange,4.7,24356,Cabernet Sauvignon
4,Unico Reserva Especial Edición,4.7,13025,Cabernet Sauvignon
0,Cabernet Sauvignon,4.8,2941,Chardonnay
1,Amarone della Valpolicella Classico Riserva,4.8,587,Chardonnay
2,Sauternes,4.7,44126,Chardonnay
3,Special Selection Cabernet Sauvignon,4.7,41236,Chardonnay
4,Pomerol,4.7,32157,Chardonnay
